## TEST 1 bev labels: Use Complex-YOLOv4 source code (bev labels don't align 100% correct)
# I guess the function map_pc2rc from bev_utils.py could bring here progress. The mapping must be in row. column format.
# https://github.com/maudzung/Complex-YOLOv4-Pytorch/blob/master/src/data_process/kitti_bev_utils.py#L164

In [ ]:
import numpy as np
import cv2
import math
import pickle

boundary = {
    "minX": 0,
    "maxX": 60,
    "minY": -30,
    "maxY": 30,
    "minZ": -2.73,
    "maxZ": 1.27
}

colors = [
    [0, 0, 0],     # Dummy
    [0, 0, 255], # Car
    [255, 0, ],   # Pedestrian
    [0, 255, 255]    # Cyclist
]

BEV_WIDTH = 640  # across y axis -25m ~ 25m
BEV_HEIGHT = 640 # across x axis 0m ~ 50m

label_lidar_path = '/home/rlab10/OpenPCDet/data/kitti/data_test_pipeline/kitti_train_dataset.pkl'
# Load gt_boxes (lidar frame)
with open(label_lidar_path, 'rb') as f:
    data = pickle.load(f)

# GT-Boxe
#gt_boxes = data[0][0]['gt_boxes'] # sample: 000000
#gt_boxes = data[1][0]['gt_boxes'] # 000003
#gt_boxes = data[2][0]['gt_boxes'] # 000007
#gt_boxes = data[3][0]['gt_boxes'] # 000009
gt_boxes = data[4][0]['gt_boxes'] # 0000010
# x, y, z, l, w, h, heading, class_idx
print('OpenPCDet format (x,y,z,l,w,h,heading,class_dx): \n', gt_boxes)
print('\n')

complex_order = [7, 0, 1, 2, 5, 4, 3, 6]
gt_boxes_reordered = gt_boxes[:, complex_order]

# cls_id, x, y, z, h, w, l, yaw
print('Complex-YOLO format (class_idx,x,y,z,h,w,l,yaw): \n', gt_boxes_reordered)
print('\n')

def build_yolo_target(labels):
    bc = boundary
    target = []
    for i in range(labels.shape[0]):
        cl, x, y, z, h, w, l, yaw = labels[i]
        # ped and cyc labels are very small, so lets add some factor to height/width
        #l = l + 0.3 # default
        #w = w + 0.2 # default
        yaw = np.pi * 2 - yaw  # default
        if (bc["minX"] < x < bc["maxX"]) and (bc["minY"] < y < bc["maxY"]):
            y1 = (y - bc["minY"]) / (bc["maxY"] - bc["minY"])
            x1 = (x - bc["minX"]) / (bc["maxX"] - bc["minX"]) 
            w1 = w / (bc["maxY"] - bc["minY"])
            l1 = l / (bc["maxX"] - bc["minX"])
            target.append([cl, y1, x1, w1, l1, math.sin(float(yaw)), math.cos(float(yaw))])

    return np.array(target, dtype=np.float32)

target = build_yolo_target(gt_boxes_reordered)
print('Build yolo target (class_idx,y,x,w,l,sin(y),cos(y)): \n',target)
print('\n')
print('Number of gt_boxes: ', len(target))

def get_corners(x, y, w, l, yaw):
    bev_corners = np.zeros((4, 2), dtype=np.float32)
    cos_yaw = np.cos(yaw)
    sin_yaw = np.sin(yaw)
    # front left
    bev_corners[0, 0] = x - w / 2 * cos_yaw - l / 2 * sin_yaw
    bev_corners[0, 1] = y - w / 2 * sin_yaw + l / 2 * cos_yaw

    # rear left
    bev_corners[1, 0] = x - w / 2 * cos_yaw + l / 2 * sin_yaw
    bev_corners[1, 1] = y - w / 2 * sin_yaw - l / 2 * cos_yaw

    # rear right
    bev_corners[2, 0] = x + w / 2 * cos_yaw + l / 2 * sin_yaw
    bev_corners[2, 1] = y + w / 2 * sin_yaw - l / 2 * cos_yaw

    # front right
    bev_corners[3, 0] = x + w / 2 * cos_yaw - l / 2 * sin_yaw
    bev_corners[3, 1] = y + w / 2 * sin_yaw + l / 2 * cos_yaw

    return bev_corners

def drawRotatedBox(img, x, y, w, l, yaw, color):
    bev_corners = get_corners(x, y, w, l, yaw)
    corners_int = bev_corners.reshape(-1, 1, 2).astype(int)
    cv2.polylines(img, [corners_int], True, color, 1)
    corners_int = bev_corners.reshape(-1, 2).astype(int)
    cv2.line(img, (corners_int[0, 0], corners_int[0, 1]), (corners_int[3, 0], corners_int[3, 1]), (255, 255, 0), 2)
   
def draw_box_in_bev(rgb_map, target):
    for j in range(len(target)):
        if (np.sum(target[j, 1:]) == 0): continue
        cls_id = int(target[j][0])
        x = target[j][1] * BEV_WIDTH
        y = target[j][2] * BEV_HEIGHT
        w = target[j][3] * BEV_WIDTH
        l = target[j][4] * BEV_HEIGHT
        yaw = np.arctan2(target[j][5], target[j][6])      
        drawRotatedBox(rgb_map, x, y, w, l, yaw, colors[cls_id])

# Test this code
#bev_image = np.zeros((BEV_HEIGHT, BEV_WIDTH, 3), dtype=np.uint8)

#bev_image = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/images for bev label test/bev_000000.png')
#bev_image = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/images for bev label test/bev_000003.png')
#bev_image = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/images for bev label test/bev_000007.png')
#bev_image = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/images for bev label test/bev_000009.png')
bev_image = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/images for bev label test/bev_000010.png')

bev_image = cv2.rotate(bev_image, cv2.ROTATE_180)
draw_box_in_bev(bev_image, target)

cv2.imshow("BEV", bev_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## TEST 2 bev labels: Here also the mapping with rows, columns could help
## Github repo: https://github.com/atanasko/ultralytics/blob/wod_obb_convert_dataset/wod_converter.py

In [ ]:
import math
import numpy as np
import pickle
import cv2

bev_width = 640
bev_height = 640
range_x = [0, 60]
range_y = [-30, 30]

def crop_bbox(x, y, size_x, size_y):
    if x - size_x / 2 < 0:
        if x < 0:
            size_x = x + size_x / 2
        if x > 0:
            size_x = (size_x / 2 - x) + size_x / 2
        x = size_x / 2
    if x + size_x / 2 > bev_width:
        if x < bev_width:
            size_x = (bev_width - x) + size_x / 2
        if x > bev_width:
            size_x = size_x / 2 - (x - bev_width)
        x = bev_width - size_x / 2
    if y - size_y / 2 < 0:
        if y < 0:
            size_y = y + size_y / 2
        if y > 0:
            size_y = (size_y / 2 - y) + size_y / 2
        y = size_y / 2
    if y + size_y / 2 > bev_height:
        if y < bev_height:
            size_y = (bev_height - y) + size_y / 2
        if y > bev_height:
            size_y = size_y / 2 - (y - bev_height)
        y = bev_height - size_y / 2

    return x, y, size_x, size_y

def rotate_point(x, y, cx, cy, theta):
    # Rotate point (x, y) around center (cx, cy) by angle (in radians)
    x_rel = x - cx
    y_rel = y - cy
    x_new = x_rel * math.cos(theta) - y_rel * math.sin(theta) + cx
    y_new = x_rel * math.sin(theta) + y_rel * math.cos(theta) + cy
    return x_new, y_new

def calculate_rotated_bbox_coordinates(cx, cy, width, height, theta):
    # Calculate the coordinates of the four corners of the rectangle
    x1 = cx - width / 2
    y1 = cy - height / 2
    x2 = cx + width / 2
    y2 = cy - height / 2
    x3 = cx + width / 2
    y3 = cy + height / 2
    x4 = cx - width / 2
    y4 = cy + height / 2

    # Rotate the four corners around the center
    x1_new, y1_new = rotate_point(x1, y1, cx, cy, theta)
    x2_new, y2_new = rotate_point(x2, y2, cx, cy, theta)
    x3_new, y3_new = rotate_point(x3, y3, cx, cy, theta)
    x4_new, y4_new = rotate_point(x4, y4, cx, cy, theta)

    return x1_new, y1_new, x2_new, y2_new, x3_new, y3_new, x4_new, y4_new

def draw_bev_bbox(bev_img, x, y, l, w, yaw, color=(0, 255, 0), thickness=1):
    # Without calculate_rotated_bbox_coordinates
    # x1 = x - size_x / 2
    # y1 = y - size_y / 2
    # x2 = x + size_x / 2
    # y2 = y - size_y / 2
    # x3 = x + size_x / 2
    # y3 = y + size_y / 2
    # x4 = x - size_x / 2
    # y4 = y + size_y / 2

     # Rotieren um den Mittelpunkt (x, y)
    # p1 = rotate_point(x1, y1, x, y, yaw)
    # p2 = rotate_point(x2, y2, x, y, yaw)
    # p3 = rotate_point(x3, y3, x, y, yaw)
    # p4 = rotate_point(x4, y4, x, y, yaw)

    # # Eckpunkte als Ganzzahlen umwandeln
    # points = np.array([
    #     [int(p1[0]), int(p1[1])],
    #     [int(p2[0]), int(p2[1])],
    #     [int(p3[0]), int(p3[1])],
    #     [int(p4[0]), int(p4[1])]
    # ], dtype=np.int32)

    corners = calculate_rotated_bbox_coordinates(x, y, l, w, yaw)

    points = np.array([[int(corners[0]), int(corners[1])],
        [int(corners[2]), int(corners[3])],
        [int(corners[4]), int(corners[5])],
        [int(corners[6]), int(corners[7])]], dtype=np.int32)

    # Polygon zeichnen
    cv2.polylines(bev_img, [points], isClosed=True, color=color, thickness=thickness)

    for point in points:
        cv2.circle(bev_img, tuple(point), radius=2, color=(255, 0, 255), thickness=-1)


# Incoming data must be ctr_x, ctr_y, ctr_z, size_x (l), size_y (w), size_z (h), heading
# My KITTI data is ctr_x, ctr_y, ctr_z, l, w, h, heading, class_idx
label_lidar_path = '/home/rlab10/OpenPCDet/data/kitti/data_test_pipeline/kitti_train_dataset.pkl'
#bev_img = np.zeros((bev_height, bev_width, 3), dtype=np.uint8)
bev_img = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/images for bev label test/bev_000010.png')
bev_img = cv2.rotate(bev_img, cv2.ROTATE_90_CLOCKWISE)

# Load gt_boxes (lidar frame)
with open(label_lidar_path, 'rb') as f:
    data = pickle.load(f)

sample_index = 4 # use this to change between 000000 - 000010
gt_boxes = data[sample_index][0]['gt_boxes']
frame = data[sample_index][0]['frame_id']
print('frame_id: ', frame)
print('Raw Bounding Boxes: ', gt_boxes)

discrete = (range_x[1] - range_x[0]) / bev_width

for box in gt_boxes:
    x, y, l, w, yaw = box[0], box[1], box[3], box[4], box[6]
    print('Extracted: ', x, y, l, w, yaw)

    x = x / discrete
    y = (-y / discrete) + bev_width / 2
    l = l / discrete
    w = w / discrete

    x, y, l, w = crop_bbox(x, y, l, w)
    print('Cropped for BEV image: ', x,y,l,w)

    draw_bev_bbox(bev_img, x, y, l, w, -yaw, color=(0, 255, 0), thickness=1)

cv2.imshow('BEV Bounding Box', bev_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## TEST 3 bev labels: BirdNet+: End-to-End 3D Object Detection in LiDAR Bird’s Eye View (works 100%)
# https://github.com/AlejandroBarrera/birdnet2/blob/5ceed811b289796d7d7420a064ecb079c80801ab/tools/convert_kitti_to_coco_rotation.py#L73

In [ ]:
import numpy as np
import pickle
import cv2
from tools.visual_utils.bev_vis_utils import draw_bbox_arrow, draw_bbox_direction, draw_bbox_keypoint


def get_bev_box_claudeai(x, y, l, w, yaw, bev_image, bvres):
    bv_image = np.array(bev_image)
    bvrows, bvcols, _ = bv_image.shape
    
    centroid = [x, y]
    print('Centroid: ', centroid)
   
    # Ursprüngliche Ecken relativ zum Zentrum
    corners = np.array([
        [-l/2., w/2.],   # links oben
        [l/2., w/2.],    # rechts oben
        [l/2., -w/2.],   # rechts unten
        [-l/2., -w/2.]   # links unten
    ])
    print('Ursprüngliche Corners (relativ zum Zentrum): \n', corners)

    # Rotationsmatrix
    yaw = - yaw
    c, s = np.cos(yaw), np.sin(yaw)
    R = np.array([[c, -s], [s, c]])
    print('Rotationsmatrix: \n', R)

    # Rotiere Corners um den Ursprung
    rotated_corners = np.dot(corners, R)
    print('Rotierte Corners (ohne Translation): \n', rotated_corners)

    # Verschiebe zurück zum Zentrum
    rotated_corners += centroid
    print('Rotierte Corners (mit Zentrum): \n', rotated_corners)

    # BEV-Koordinatentransformation
    x1 = bvcols / 2 + (-rotated_corners[:, 1]) / bvres
    y1 = bvrows - rotated_corners[:, 0] / bvres
    
    transformed_corners = np.column_stack((x1, y1))
    print('Transformierte BEV-Corners: \n', transformed_corners)

    return transformed_corners
   

box_colormap = [
    [255, 255, 255], # not assigned
    [0, 255, 0], # Car Green
    [255, 0, 255], # Pedestrian Violet
    [0, 255, 255], # Cyclist Yellow
]

class_names = { 
    1: 'Car', 
    2: 'Ped',
    3: 'Cyc'
}

# ==============================================================================
# DRAWS A ROTATED 2D-BOUNDING BOX ONTO A BEV IMAGE (BOX IN LiDAR FRAME REQUIRED)
# ==============================================================================
def get_rot_bevbox(x, y, l, w, yaw, cls, bev_img, bev_res=0.1):
    """
    source: https://github.com/AlejandroBarrera/birdnet2
    other worthwhile sources: https://github.com/maudzung/Complex-YOLOv4-Pytorch/blob/master/src/data_process/kitti_bev_utils.py, 
    https://github.com/spmallick/learnopencv/blob/master/3D-LiDAR-Object-Detection/sfa/data_process/kitti_bev_utils.py#L87
    """
    # Convert BEV image to array and extract dimensions
    bev_img = np.array(bev_img)
    bvrows, bvcols, _ = bev_img.shape
    # Define object's centroid, adjust rotation angle
    centroid = [x, y]
    
    if cls == 1: # Car
        # CAR
        l = l + 0.4
        w = w + 0.4
    elif cls == 2: # Pedestrian
        # https://github.com/maudzung/Complex-YOLOv4-Pytorch/blob/master/src/data_process/kitti_bev_utils.py
        # PEDESTRIAN; Ped. and Cyc. labels are very small, so lets add some factor to height/width
        l = l + 0.3
        w = w + 0.3
    elif cls == 3: # Cyclist
        # CYCLIST
        l = l + 0.3
        w = w + 0.3

    yaw = -yaw # Invert yaw from LiDAR frame to Image frame

    # Calculate the initial coordinates of the object's four corners (relative to the centroid)
    corners = np.array([[centroid[0] - l/2., centroid[1] + w/2.], # Top-left
                        [centroid[0] + l/2., centroid[1] + w/2.], # Top-right
                        [centroid[0] + l/2., centroid[1] - w/2.], # Bottom-right
                        [centroid[0] - l/2., centroid[1] - w/2.]]) # Bottom-left

    # Compute rotation matrix for yaw angle
    cos, sin = np.cos(yaw), np.sin(yaw)
    """
    2D-Rotation matrix
    [x'] = [cos(yaw) -sin(yaw)] * [x]
    [y']   [sin(yaw)  cos(yaw)]   [y]
    """
    R = np.array([[cos, -sin], [sin, cos]])

    # Rotate all corners around the centroid
    rotated_corners = np.dot(corners-centroid, R) + centroid

    # Convert the world coordinates to BEV image coordinates
    x1, x2, x3, x4 = bvcols / 2 + (-rotated_corners[:, 1]) / bev_res  # world y -> image x (u)
    y1, y2, y3, y4 = bvrows - rotated_corners[:, 0] / bev_res         # world x -> image y (v)

    x1, x2, x3, x4 = x3, x4, x1, x2
    y1, y2, y3, y4 = y3, y4, y1, y2
    
    # Remove bev labels for objects outside the BEV image
    is_fully_visible = not (
        (x1 < 0 or x1 >= bvcols) or
        (x2 < 0 or x2 >= bvcols) or
        (x3 < 0 or x3 >= bvcols) or
        (x4 < 0 or x4 >= bvcols) or
        (y1 < 0 or y1 >= bvrows) or
        (y2 < 0 or y2 >= bvrows) or
        (y3 < 0 or y3 >= bvrows) or
        (y4 < 0 or y4 >= bvrows)
    )
    
    # Calculate ROI of box
    x_min = max(0, int(min(x1, x2, x3, x4)))
    x_max = min(bvcols - 1, int(max(x1, x2, x3, x4)))
    y_min = max(0, int(min(y1, y2, y3, y4)))
    y_max = min(bvrows - 1, int(max(y1, y2, y3, y4)))

    # Remove objects with fewer than 3 points in the box
    roi = bev_img[y_min:y_max, x_min:x_max]
    """
    (x1,y1)------(x2,y1)
     |              |
     |      ROI     |
     |              |
    (x1,y2)------(x2,y2)
    """
    nonzero = np.count_nonzero(np.sum(roi, axis=2))
    if nonzero < 3:  # Detection is considered unreliable with fewer than 3 points
        return -1, -1, -1, -1
   
    if is_fully_visible:
        return x1, y1, x2, y2, x3, y3, x4, y4, box_colormap[int(cls)], centroid 
        #return x1, x2, x3, x4, y1, y2, y3, y4, box_colormap[int(cls)], centroid # Return the coordinates of the four corners of the rotated bounding box in BEV image space
    else:
        return np.array([-1, -1, -1, -1]) # # Indicates the box should not be drawn (out of bounds)
    

def draw_lbl_and_score(image, class_name, conf_score=None, centroid = (0, 0), bev_res=0.1, color=(255, 255, 255), font_scale=0.5, thickness=1):
    score = f'{conf_score:.2f}' if conf_score is not None else '-.--'

    ctr_img_x = int(image.shape[1] / 2 + (-centroid[1]) / bev_res)  
    ctr_img_y = int(image.shape[0] - centroid[0] / bev_res) 

    (lbl_width, lbl_height), _ = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_DUPLEX, font_scale, thickness)
    (scr_width, scr_height), _ = cv2.getTextSize(score, cv2.FONT_HERSHEY_DUPLEX, font_scale, thickness)

    c_x, c_y = ctr_img_x, ctr_img_y
    lbl_x = int(c_x - lbl_width / 2) + 22  
    lbl_y = int(c_y + lbl_height / 2) + 15 
    lbl_x = max(0, min(lbl_x, image.shape[1] - lbl_width))
    lbl_y = max(0, min(lbl_y, image.shape[0] - lbl_height))

    scr_x = int(c_x + 38) 
    scr_y = int(c_y + 22)
    scr_x = max(0, min(scr_x, image.shape[1] - scr_width))
    scr_y = max(0, min(scr_y, image.shape[0] - scr_height))
  
    # Label
    cv2.putText(image, class_name, (lbl_x, lbl_y), cv2.FONT_HERSHEY_DUPLEX, font_scale, color, thickness, cv2.LINE_AA)
    # Score
    cv2.putText(image, str(score), (scr_x, scr_y), cv2.FONT_HERSHEY_DUPLEX, 0.4, (255,255,255), thickness, cv2.LINE_AA)


def main (mode=str):

    if mode == 'train':
        #label_lidar_path = '/home/rlab10/OpenPCDet/data/kitti/data_test_pipeline/visualizer/kitti_train_dataset.pkl'
        label_lidar_path = '/home/rlab10/OpenPCDet/data/kitti/kitti_train_dataset.pkl'
        #bev_img = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/images for bev label test/bev_000000.png')
        #bev_img = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/train/imgs/bev_000000_1.png')
        bev_img = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/train/imgs/bev_000064_4.png')

        with open(label_lidar_path, 'rb') as f:
            data = pickle.load(f)

        sample_index = 155 # use this to change between 000000 - 000010
        augment_index = 4 # use this to change the augmentation
        gt_boxes = data[sample_index][augment_index]['gt_boxes']

    elif mode == 'val':
        label_lidar_path = '/home/rlab10/OpenPCDet/data/kitti/data_test_pipeline/kitti_val_dataset.pkl'
        bev_img = cv2.imread('/home/rlab10/OpenPCDet/lidar2bev/bev_images/val/imgs/bev_val_000001.png')

        with open(label_lidar_path, 'rb') as f:
            data = pickle.load(f)

        sample_index = 0 # use this to change between 000001 - 000006
        gt_boxes = data[sample_index]['annos']['gt_boxes_lidar']

    bev_res = 0.1

    for box in gt_boxes:
    
        x, y, l, w, yaw, cls = [box[i] for i in [0, 1, 3, 4, 6, 7]]

        result = get_rot_bevbox(x, y, l, w, yaw, cls, bev_img, bev_res=bev_res)

        if isinstance(result, np.ndarray) and np.any(result == -1):
            continue

        x1, y1, x2, y2, x3, y3, x4, y4, bbox_color, centroid  = result
        polygon = np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], dtype=np.int32).reshape((-1, 1, 2))
        cv2.polylines(bev_img, [polygon], isClosed=True, color=bbox_color, thickness=1)

        score = 0.94
        draw_lbl_and_score(bev_img, class_names.get(cls, 'Unknown'), conf_score=score, centroid=centroid, color = bbox_color)


        #for x, y in [(x1, y1)]:
        #    cv2.circle(bev_img, (int(x), int(y)), radius=3, color=(0, 0, 255), thickness=-1)  # Red points
      
        #draw_bbox_direction(bev_img, np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]]))
        #draw_bbox_arrow(bev_img, centroid=centroid, yaw=yaw, cls=cls, bev_res=bev_res, color=bbox_color)
        #draw_bbox_keypoint(bev_img, centroid=centroid, cls=cls, bev_res=bev_res)
        

    cv2.imshow('BEV Image', bev_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main('train')
    #main('val')

In [ ]:
import pickle

label_lidar_path = '/home/rlab10/OpenPCDet/data/kitti/kitti_train_dataset.pkl'

with open(label_lidar_path, 'rb') as f:
            data = pickle.load(f)

data[0][0]

## TEST 4 bev labels: OpenPCDet intern functions (NOT TESTED)
https://blog.csdn.net/W1995S/article/details/115486685

In [2]:
import torch
from pcdet.ops.iou3d_nms.iou3d_nms_utils import boxes_iou3d_gpu
from pcdet.utils.box_utils import boxes3d_nearest_bev_iou


def assign_targets_single(self, anchors, gt_boxes, gt_classes, matched_threshold=0.6, unmatched_threshold=0.45):
   num_anchors = anchors.shape[0]
   num_gt = gt_boxes.shape[0]

   labels = torch.ones((num_anchors,), dtype=torch.int32, device=anchors.device) * -1
   gt_ids = torch.ones((num_anchors,), dtype=torch.int32, device=anchors.device) * -1

   if len(gt_boxes) > 0 and anchors.shape[0] > 0:
      anchor_by_gt_overlap = boxes_iou3d_gpu(anchors[:, 0:7], gt_boxes[:, 0:7]) \
        if self.match_height else boxes3d_nearest_bev_iou(anchors[:, 0:7], gt_boxes[:, 0:7])

## TEST 1 yolo obb health check: https://github.com/ultralytics/ultralytics/issues/13799

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

bev_val_img = '/home/rlab10/OpenPCDet/lidar2bev/bev_images/val/imgs/bev_val_000006.png'
bev_val_lbl = '/home/rlab10/OpenPCDet/lidar2bev/bev_images/val/labels/bev_val_000006.txt'

bev_train_img = '/home/rlab10/OpenPCDet/lidar2bev/bev_images/train/imgs/bev_000003_4.png'
bev_train_lbl = '/home/rlab10/OpenPCDet/lidar2bev/bev_images/train/labels/bev_000003_4.txt'


def draw_obb(image_path, label_path):
    image = cv2.imread(image_path)

    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            cls, points = int(parts[0]), list(map(float, parts[1:9]))
            points = [(int(points[i] * image.shape[1]), int(points[i+1] * image.shape[0])) for i in range(0, len(points), 2)]
            points = np.array(points, np.int32).reshape((-1, 1, 2))
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=1)

    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    #cv2.imshow('BEV Visualization', image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

#draw_obb(bev_val_img, bev_val_lbl)
draw_obb(bev_train_img, bev_train_lbl)